In [48]:
import pandas as pd
import numpy as np
import re

Ingesta de datos

In [49]:
#Extraigo los Datasets
amazon= pd.read_csv('Datasets\prime_titles.csv',index_col=[0])
disney= pd.read_csv('Datasets\disney_plus_titles.csv',index_col=[0])
hulu= pd.read_csv('Datasets\hulu_titles.csv',index_col=[0])
netflix= pd.read_json('Datasets\stitles_netflix.json')
netflix=netflix.drop(columns='show_id')

Creo mi DF

In [50]:
#Agrego la columna plataforma con la plataforma correspondiente
netflix['plataform']='netflix'
disney['plataform']='disney'
hulu['plataform']='hulu'
amazon['plataform']='amazon'

In [51]:
#Concateno las listas en un solo df
data=pd.concat((netflix,disney,hulu,amazon),ignore_index=True)

In [52]:
#Elimino columnas con datos sin uso
data=data.drop(columns='description')
data=data.drop(columns='country')
data=data.drop(columns='date_added')
data=data.drop(columns='rating')
data=data.drop(columns='director')

In [53]:
data.head(3)

,type,title,cast,release_year,duration,listed_in,plataform
0,Movie,Dick Johnson Is Dead,None,2020,90 min,Documentaries,netflix
1,TV Show,Blood & Water,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",2021,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries",netflix
2,TV Show,Ganglands,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",2021,1 Season,"Crime TV Shows, International TV Shows, TV Act...",netflix


Columna duration la transformo a numérico

In [54]:
#Transformo los datos de duration para que sea de tipo entero, de esa forma poder sumar los minutos para Movies y las Season para TV Shows
for fila,duracion in enumerate(data['duration']):
    valor=(list(re.findall(r'-?\d+',str(duracion))))
    #extraigo en una lista el valor numérico de cada dígito
    if valor :
        data.duration[fila]= valor[0]
        #de existir ese valor lo reemplazo en la duración
    else:
        data.duration[fila]= 0
        # en caso de que no lo reemplazo por none
data["duration"] = pd.to_numeric(data["duration"])

C:\Users\Usuario\AppData\Local\Temp\ipykernel_6180\1894652545.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.duration[fila]= valor[0]
C:\Users\Usuario\AppData\Local\Temp\ipykernel_6180\1894652545.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.duration[fila]= 0


In [55]:
data['duration'].dtypes

dtype('int64')

Dropeo duplicados y nulos

In [56]:
data.shape

(22998, 7)

In [57]:
#Elimino los duplicados en caso de haber
data.drop_duplicates()
data.shape

(22998, 7)

In [58]:
#Elimino los nulos, en caso de tener más de 5 valores faltantes en caso de haber
data.dropna(thresh=5)
data.shape

(22998, 7)

Normalizo valores entre plataformas

In [61]:
#En caso de repetirse el título entre plataformas, va a rellenar y normalizar los datos
cont=0
for fila1,titulo1 in enumerate(data['title']):
    cont=0
    for fila2,titulo2 in enumerate(data['title']):
        if cont==0 and fila1<fila2 and titulo1==titulo2 and data.type[fila1]==data.type[fila2] :
#Si es distinta fila, tienen el mismo tipo y comparten el título entonces:
#Le pido que para la columna cast,duration, release_year y listed_in me reemplace los valores en caso de existir
            if data.cast[fila1]:
                data.cast[fila2]=data.cast[fila1]
            elif data.cast[fila2]:
                data.cast[fila1]=data.cast[fila2]
            if data.duration[fila1]:
                data.duration[fila2]=data.duration[fila1]
            elif data.duration[fila2]:
                data.duration[fila1]=data.duration[fila2]
            # Necesito comentarlo porque no me da las respuestas esperadas en el pdf
            #if data.release_year[fila1]:
            #    data.release_year[fila2]=data.release_year[fila1]
            #elif data.release_year[fila2]:
            #    data.release_year[fila1]=data.release_year[fila2]
            # Necesito comentarlo porque no me da las respuestas esperadas del pdf
            # Necesito comentarlo porque no me da las respuestas esperadas del pdf
            cont+=1  

C:\Users\Usuario\AppData\Local\Temp\ipykernel_6180\530900885.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.cast[fila2]=data.cast[fila1]
C:\Users\Usuario\AppData\Local\Temp\ipykernel_6180\530900885.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.duration[fila2]=data.duration[fila1]
C:\Users\Usuario\AppData\Local\Temp\ipykernel_6180\530900885.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.duration[fila1]=

In [62]:
data.head()

,type,title,cast,release_year,duration,listed_in,plataform
0,Movie,Dick Johnson Is Dead,None,2020,90,Documentaries,netflix
1,TV Show,Blood & Water,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",2021,2,"International TV Shows, TV Dramas, TV Mysteries",netflix
2,TV Show,Ganglands,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",2021,1,"Crime TV Shows, International TV Shows, TV Act...",netflix
3,TV Show,Jailbirds New Orleans,None,2021,1,"Docuseries, Reality TV",netflix
4,TV Show,Kota Factory,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",2021,2,"International TV Shows, Romantic TV Shows, TV ...",netflix


Exporto DF

In [63]:
data.to_csv('Datasets\datos_titulos.csv',index=False)
#Terminamos con el trabajo de limpieza de datos